In [1]:
import time
import pandas as pd
from content_cleaning import *

In [2]:
# 初始化处理器
processor = XHSMixedLanguageProcessor(cache_size=2000, max_workers=12)

# # 示例笔记
# sample_notes = [
#     "今天去了Dallas的鲜芋仙 #美食打卡# 这家MeetFresh真的超级好吃！The taro balls were amazing! 强烈推荐大家尝试～",
#     "新开的台湾甜品店@小红书美食博主 服务态度nice，芋圆Q弹，仙草冻很香 #DFW美食# [笑哭R] 2001 Coit Rd真的很方便",
#     "约上闺蜜一起去吃甜品，牛奶冰+芋圆组合👍 The dessert was incredibly refreshing on such a hot day! https://xiaohongshu.com/...",
#     "今日份打卡：鲜芋仙 Park Pavillion Center，人均$15左右，店内环境整洁，服务态度很好，definitely worth the price!",
# ]

# processed_text = processor.process_text(sample_notes[0], enable_translation=True)
# print("单条处理结果:", processed_text)
# processed_texts = processor.batch_process(sample_notes)
# print("\n批量处理结果:")
# for i, text in enumerate(processed_texts):
#     print(f"{i+1}. {text}")

# Load the cooked contents data
cont = pd.read_json('..\..\Data\processed\contents_cooked.json')

# combine the title and note_body into a single string
def process_text(note):
    return note['title'] + ' ' + note['note_body']

# Apply the function to the DF
cont['text'] = cont.apply(process_text, axis=1).astype(str)

# Apply the batch processing function to the DF.text column
start_time = time.time()
processed_texts = processor.batch_process(cont['text'].tolist(), enable_translation=True)
end_time = time.time()

print(f"批量处理耗时: {end_time - start_time:.2f}秒")

# save the processed texts to a new column in the DF
cont['semantic_proc_text'] = processed_texts
# remove the original text column
cont.drop(columns=['text'], inplace=True)
# save the processed DF to a new JSON file
cont.to_json('..\..\Data\processed\contents_cooked_semantic.json', orient='records', lines=True, force_ascii=False)


批量处理耗时: 432.16秒


In [3]:
cont.columns

Index(['note_id', 'user_id', 'title', 'note_body', 'tag_list', 'image_count',
       'content_type_video', 'hot_note', 'post_time', 'last_update_time',
       'scraped_time', 'elapsed_time', 'liked_count', 'collected_count',
       'comment_count', 'share_count', 'interaction_count',
       'semantic_proc_text'],
      dtype='object')

In [4]:
processor.get_stats()

{'cache_size': 1965,
 'cache_capacity': 2000,
 'cache_hits': 176,
 'translation_requests': 16883,
 'cache_hit_rate': 0.01042468755552923}